In [326]:
import matplotlib.pyplot as plt
%matplotlib inline

In [45]:
import pandas as pd
import io
import requests
import os
import numpy as np
# import schedule
import time

----------
ETL

----------

https://www.data.gouv.fr/fr/datasets/indicateurs-de-suivi-de-lepidemie-de-covid-19/
https://www.data.gouv.fr/fr/datasets/donnees-relatives-aux-resultats-des-tests-virologiques-covid-19/

In [38]:
import pandas as pd
import io
import requests
import os
import numpy as np
import schedule
import time



def job():
    
    def extract(local=True):

        if local:
            df = pd.read_csv("../data/chiffres-cles.csv")
            quatre_taux = pd.read_csv("../data/table-indicateurs-open-data-dep-serie.csv")
        else: 
            url = "https://github.com/opencovid19-fr/data/raw/master/dist/chiffres-cles.csv"
            s = requests.get(url).content
            df = pd.read_csv(io.StringIO(s.decode('utf-8')))

            url= "https://www.data.gouv.fr/fr/datasets/r/4acad602-d8b1-4516-bc71-7d5574d5f33e"
            s=requests.get(url).content
            quatre_taux = pd.read_csv(io.StringIO(s.decode('utf-8')))

        dpt = pd.read_csv("../data/departements-region.csv")
        reg = pd.read_csv("../data/regions-france.csv")
        pop = pd.read_csv("../data/estim_pop.csv")


        df["date"] = df["date"].replace("_", "-", regex=True)
        df["date"] = pd.to_datetime(df["date"])

        df["dep_code"] = df[(df['maille_code'].str.startswith('DEP')) | (df['maille_code'].str.startswith('COM'))]["maille_code"]\
        .replace(".*-", "", regex=True)
        df["reg_code"] = df[df['maille_code'].str.startswith('REG')]["maille_code"]\
        .replace(".*-", "", regex=True).astype("int64")

        quatre_taux["extract_date"] = pd.to_datetime(quatre_taux["extract_date"])


        geo = dpt.merge(reg, right_on= 'nom_region', left_on='region_name')\
        .rename(columns = {'num_dep':'code_dep'})\
        .drop(columns=["nom_region"])[['code_region', 'region_name', 'code_dep', 'dep_name']]

        pop[["2020", "2021"]] = pop[["2020", "2021"]].replace("\s", "", regex=True).astype(int)

        return [df, quatre_taux, geo, pop]


    def transform(dataframe):

        df, quatre_taux, geo, pop = dataframe[0], dataframe[1], dataframe[2], dataframe[3]

        df_clean = df.merge(geo, right_on="code_dep", left_on='dep_code')[['date', 'code_region', 'region_name', 'code_dep',
               'dep_name', 'cas_confirmes',
               'cas_ehpad', 'cas_confirmes_ehpad', 'cas_possibles_ehpad', 'deces',
               'deces_ehpad', 'reanimation', 'hospitalises',
               'nouvelles_hospitalisations', 'nouvelles_reanimations', 'gueris',
               'depistes', 'source_nom', 'source_url', 'source_archive', 'source_type']]

        rs = []
        for ind, row in df_clean.iterrows():   #[["deces","gueris", 'reanimation', 'hospitalises']]
            if row["date"].year == 2020:
                effectif = pop.loc[pop['code_dpt'] == row['code_dep'], "2020"]
            else: 
                effectif = pop.loc[pop['code_dpt'] == row['code_dep'], "2021"]
            dc = row["deces"] / effectif * 100000
            gu = row["gueris"] / effectif * 100000
        #     rea = row["reanimation"] / effectif * 100000
            hos = row["hospitalises"] / effectif * 100000
        #     tot= dc.values[0] + gu.values[0] + hos.values[0]

            rs.append([row["date"], row["region_name"], row["code_dep"], row["dep_name"], dc.values[0], gu.values[0], hos.values[0]])
        col = ['date', 'region_name', 'dep_code', 'dep_name', 'deces', 'gueris', 'symptomatiques']            


        final = pd.DataFrame(rs, columns = col)\
        .merge(quatre_taux[["extract_date",'departement', 'tx_incid']], left_on=['date','dep_code'], right_on=["extract_date",'departement'],suffixes=('_x', '_y'))\
        .drop(columns=["extract_date",'departement'])#\
    #     .rename(columns= {"tx_incid" : "susceptibles"})

        final["asymptomatiques"] = final["tx_incid"] - final['symptomatiques']

    #     start_date = final[final['asymptomatiques']>0].sort_values('date').iloc[0,0]
        final = final[final["date"].duplicated()]
        final.loc[(final['asymptomatiques']<0), "asymptomatiques"] = float('NaN')
        t = final.groupby("dep_name").size()
        to_drop = t[t<50].index
        final = final[~final["dep_name"].isin(to_drop)]
    #     final = final.loc[(final['date'] >= start_date)  , : ]

        rs=[]
        for name, data in final.groupby('dep_name'):
            data = data.reset_index()
    #         print(name)
            data.sort_values('date', inplace=True)
            if data['tx_incid'].isnull().loc[0]:
                data.loc[0,'tx_incid'] = 0
            if data['deces'].isnull().loc[0]:
                data.loc[0,'deces'] = 0 
            if data['gueris'].isnull().loc[0]:
                data.loc[0,'gueris'] = 0 
            if data['symptomatiques'].isnull().loc[0]:
                data.loc[0,'symptomatiques'] = 0 
            if data['asymptomatiques'].isnull().loc[0]:
                data.loc[0,'asymptomatiques'] = 0 
                data = data.set_index('date').interpolate(method='linear')
                data = data[data.index.duplicated(keep="last")].drop(columns='index')
                rs.append(data)
        final = pd.concat(rs)

        return final

    def load(file, path = "../data/df_clean.csv"):
        if path = "../data/df_clean.csv":
            os.rename("../data/df_clean.csv", "../data/_old_df_clean.csv")
        file.to_csv(path)

    e = extract(local=False)
    df = transform(e)
    load(df)
    
schedule.every().day.at("02:00").do(job)

if __name__== '__main__':
    while True:
    schedule.run_pending()
    time.sleep(1)

IndentationError: unexpected indent (<ipython-input-38-bd4bda15555c>, line 13)

In [47]:
def extract(local=True):

    if local:
        df = pd.read_csv("../data/chiffres-cles.csv")
        quatre_taux = pd.read_csv("../data/table-indicateurs-open-data-dep-serie.csv")
    else: 
        url = "https://github.com/opencovid19-fr/data/raw/master/dist/chiffres-cles.csv"
        s = requests.get(url).content
        df = pd.read_csv(io.StringIO(s.decode('utf-8')))

        url= "https://www.data.gouv.fr/fr/datasets/r/4acad602-d8b1-4516-bc71-7d5574d5f33e"
        s=requests.get(url).content
        quatre_taux = pd.read_csv(io.StringIO(s.decode('utf-8')))

    dpt = pd.read_csv("../data/departements-region.csv")
    reg = pd.read_csv("../data/regions-france.csv")
    pop = pd.read_csv("../data/estim_pop.csv")


    df["date"] = df["date"].replace("_", "-", regex=True)
    df["date"] = pd.to_datetime(df["date"])

    df["dep_code"] = df[(df['maille_code'].str.startswith('DEP')) | (df['maille_code'].str.startswith('COM'))]["maille_code"]\
    .replace(".*-", "", regex=True)
    df["reg_code"] = df[df['maille_code'].str.startswith('REG')]["maille_code"]\
    .replace(".*-", "", regex=True).astype("int64")

    quatre_taux["extract_date"] = pd.to_datetime(quatre_taux["extract_date"])


    geo = dpt.merge(reg, right_on= 'nom_region', left_on='region_name')\
    .rename(columns = {'num_dep':'code_dep'})\
    .drop(columns=["nom_region"])[['code_region', 'region_name', 'code_dep', 'dep_name']]

    pop[["2020", "2021"]] = pop[["2020", "2021"]].replace("\s", "", regex=True).astype(int)

    return [df, quatre_taux, geo, pop]


def transform(dataframe):

    df, quatre_taux, geo, pop = dataframe[0], dataframe[1], dataframe[2], dataframe[3]

    df_clean = df.merge(geo, right_on="code_dep", left_on='dep_code')[['date', 'code_region', 'region_name', 'code_dep',
           'dep_name', 'cas_confirmes',
           'cas_ehpad', 'cas_confirmes_ehpad', 'cas_possibles_ehpad', 'deces',
           'deces_ehpad', 'reanimation', 'hospitalises',
           'nouvelles_hospitalisations', 'nouvelles_reanimations', 'gueris',
           'depistes', 'source_nom', 'source_url', 'source_archive', 'source_type']]

    rs = []
    for ind, row in df_clean.iterrows():   #[["deces","gueris", 'reanimation', 'hospitalises']]
        if row["date"].year == 2020:
            effectif = pop.loc[pop['code_dpt'] == row['code_dep'], "2020"]
        else: 
            effectif = pop.loc[pop['code_dpt'] == row['code_dep'], "2021"]
        dc = row["deces"] / effectif * 100000
        gu = row["gueris"] / effectif * 100000
    #     rea = row["reanimation"] / effectif * 100000
        hos = row["hospitalises"] / effectif * 100000
    #     tot= dc.values[0] + gu.values[0] + hos.values[0]

        rs.append([row["date"], row["region_name"], row["code_dep"], row["dep_name"], dc.values[0], gu.values[0], hos.values[0]])
    col = ['date', 'region_name', 'dep_code', 'dep_name', 'deces', 'gueris', 'symptomatiques']            


    final = pd.DataFrame(rs, columns = col)\
    .merge(quatre_taux[["extract_date",'departement', 'tx_incid']], left_on=['date','dep_code'], right_on=["extract_date",'departement'],suffixes=('_x', '_y'))\
    .drop(columns=["extract_date",'departement'])#\
#     .rename(columns= {"tx_incid" : "susceptibles"})

    final["asymptomatiques"] = final["tx_incid"] - final['symptomatiques']

#     start_date = final[final['asymptomatiques']>0].sort_values('date').iloc[0,0]
    final = final[final["date"].duplicated()]
    final.loc[(final['asymptomatiques']<0), "asymptomatiques"] = float('NaN')
    t = final.groupby("dep_name").size()
    to_drop = t[t<50].index
    final = final[~final["dep_name"].isin(to_drop)]
#     final = final.loc[(final['date'] >= start_date)  , : ]

    rs=[]
    for name, data in final.groupby('dep_name'):
        data = data.reset_index()
#         print(name)
        data.sort_values('date', inplace=True)
        if data['tx_incid'].isnull().loc[0]:
            data.loc[0,'tx_incid'] = 0
        if data['deces'].isnull().loc[0]:
            data.loc[0,'deces'] = 0 
        if data['gueris'].isnull().loc[0]:
            data.loc[0,'gueris'] = 0 
        if data['symptomatiques'].isnull().loc[0]:
            data.loc[0,'symptomatiques'] = 0 
        if data['asymptomatiques'].isnull().loc[0]:
            data.loc[0,'asymptomatiques'] = 0 
            data = data.set_index('date').interpolate(method='linear')
            data = data[data.index.duplicated(keep="last")].drop(columns='index')
            rs.append(data)
    final = pd.concat(rs)

    return final

def load(file, path = "../data/df_clean.csv"):
    if path == "../data/df_clean.csv":
        os.rename("../data/df_clean.csv", "../data/_old_df_clean.csv")
    file.to_csv(path)

e = extract(local=False)
df = transform(e)
#load(df)


/home/danpdvn/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [48]:
final = df.copy()

In [58]:
deriv_final = pd.concat([final.loc[:,:'dep_name'],final.loc[:, 'deces':].diff().fillna(0)], axis=1)

-------
SEAIR

-------

In [73]:
deriv_final.groupby(['dep_name'])['tx_incid'].apply(lambda x: x.iloc[0])

dep_name
Ain                        0.000000
Aisne                     -0.379385
Allier                    -0.925217
Alpes-Maritimes            0.000000
Alpes-de-Haute-Provence   -0.242623
                             ...   
Var                        0.000000
Vaucluse                  -0.011803
Vendée                     0.090287
Vienne                    -0.385000
Vosges                    -0.230538
Name: tx_incid, Length: 83, dtype: float64

In [ ]:
alpha = data.iloc[0, -4] / data.iloc[0, -2]
beta = data.iloc[0, -2] * 1000
gamma1 = data.iloc[0, -4] * data.iloc[0, -5]*(10**6)
gamma2 = data.iloc[0, -1] * data.iloc[0, -5]*(10**6)
delta = gamma1 / gamma2
theta = data
mu = 0

In [ ]:
#SEAIRD
import numpy as np

def params(data):
    alpha = data.iloc[0, -4] / data.iloc[0, -2]
    beta = data.iloc[0, -2] * 1000
    gamma1 = data.iloc[0, -4] * data.iloc[0, -5]*(10**6)
    gamma2 = data.iloc[0, -1] * data.iloc[0, -5]*(10**6)
    delta = gamma1 / gamma2
    theta = data.iloc[0, 4] * 1000
    mu = 0

def model(ini, time_step, params):
    Y = np.zeros(6) #column vector for the state variables
    S, E, I, A, R, D = ini
    alpha = params[0]
    beta = params[1]
    delta = params[2]
    gamma1 = params[3]
    gamma2 =params[4]
    theta = params[5]
    mu = 0

    Y[0] = - ( (beta*S*I + delta*A) / (S+E+I+A+R) ) - S + 100000 #S
    Y[1] = ( (beta*S*I + delta*A) / (S+E+I+A+R) ) - (mu +1)*E #E
    Y[2] = alpha*mu*E -(gamma1 + theta + 1)*I #I
    Y[3] = (1-alpha)*mu*E -(gamma + 1)*A #A
    Y[4] = gamma1*I + gamma2*A - R #R
    Y[5] = theta*I #D

    return Y

def x0fcn(data):
    data = data.iloc[0,:]
    E0 = data["tx_incid"]
    A0 = data["asymptomatiques"]
    I0 = data["tx_incid"] - data["asymptomatiques"]
    R0 = data["gueris"]
    D0 = data["deces"]
    S0 = 100000.0 - E0 - A0 - I0 - R0 - D0
    X0 = [S0, E0, A0, I0, R0, D0]

    return X0


def yfcn(res, params):
    return res[:,1]*params[2]

In [ ]:
from scipy.stats import poisson
from scipy.stats import norm

from scipy.integrate import odeint as ode

def NLL(params, data, times): #negative log likelihood
    params = np.abs(params)
    # 	data = np.array(data)
    res = ode(model, sir_ode.x0fcn(params,data), times, args=(params,))
    y = yfcn(res, params)
    nll = sum(y) - sum(data*np.log(y))
    # note this is a slightly shortened version--there's an additive constant term missing but it 
    # makes calculation faster and won't alter the threshold. Alternatively, can do:
    # nll = -sum(np.log(poisson.pmf(np.round(data),np.round(y)))) # the round is b/c Poisson is for (integer) count data
    # this can also barf if data and y are too far apart because the dpois will be ~0, which makes the log angry

    # ML using normally distributed measurement error (least squares)
    # nll = -sum(np.log(norm.pdf(data,y,0.1*np.mean(data)))) # example WLS assuming sigma = 0.1*mean(data)
    # nll = sum((y - data)**2)  # alternatively can do OLS but note this will mess with the thresholds 
    #                             for the profile! This version of OLS is off by a scaling factor from
    #                             actual LL units.
    return nll

In [ ]:
#minimum fischer matrix
def minifisher (times, params, data, delta = 0.001):
    #params = np.array(params)
    listX = []
    params_1 = np.array (params)
    params_2 = np.array (params)
    for i in range(len(params)):
        params_1[i] = params[i] * (1+delta)
        params_2[i]= params[i] * (1-delta)

        res_1 = ode(model, x0fcn(params_1,data), times, args=(params_1,))
        res_2 = ode(model, x0fcn(params_2,data), times, args=(params_2,))
        subX = (yfcn(res_1, params_1) - yfcn(res_2, params_2)) / (2 * delta * params[i])
        listX.append(subX.tolist())
    X = np.matrix(listX)
    FIM = np.dot(X, X.transpose())
    return FIM

In [ ]:
def proflike (params, profindex, cost_func, times, data, perrange = 0.5, numpoints = 10):
    profrangedown = np.linspace(params[profindex], params[profindex] * (1 - perrange), numpoints).tolist()
    profrangeup = np.linspace(params[profindex], params[profindex] * (1 + perrange), numpoints).tolist()[1:] #skip the duplicated values
    profrange = [profrangedown, profrangeup]
    currvals = []
    currparams = []
    currflags = []

    def profcost (fit_params, profparam, profindex, data, times, cost_func):
        paramstest = fit_params.tolist()
        paramstest.insert(profindex, profparam)
        return cost_func (paramstest, data, times)

    fit_params = params.tolist() #make a copy of params so we won't change the origianl list
    fit_params.pop(profindex)
    print('Starting profile...')
    for i in range(len(profrange)):
        for j in profrange[i]:
            print(i, j)
            optimizer = optimize.minimize(profcost, fit_params, args=(j, profindex, data, times, cost_func), method='Nelder-Mead')
            fit_params = np.abs(optimizer.x).tolist() #save current fitted params as starting values for next round
            #print optimizer.fun
            currvals.append(optimizer.fun)
            currflags.append(optimizer.success)
            currparams.append(np.abs(optimizer.x).tolist())

    #structure the return output
    profrangedown.reverse()
    out_profparam = profrangedown+profrangeup
    temp_ind = range(len(profrangedown))
    temp_ind.reverse()
    out_params = [currparams[i] for i in temp_ind]+currparams[len(profrangedown):]
    out_fvals = [currvals[i] for i in temp_ind]+currvals[len(profrangedown):]
    out_flags = [currflags[i] for i in temp_ind]+currflags[len(profrangedown):]
    output = {'profparam': out_profparam, 'fitparam': np.array(out_params), 'fcnvals': out_fvals, 'convergence': out_flags}
    return output
